In [14]:
import os
import math
import numpy as np
import string

import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe
import openrtdynamics2.code_generation_helper as cgh
from openrtdynamics2.targets import TargetTemplate
from openrtdynamics2.manifest_import import get_all_inputs, get_all_outputs

from textwrap import indent


# An example for generating code in form of a Simulink S-function

In [31]:
class CppSimulinkSFunction(TargetTemplate):
    """
        generate a simple main.cpp programm
    """

    def __init__(self, enable_tracing=False, sfun_name : str = 'sfunction'):
        TargetTemplate.__init__(self, enable_tracing)
        self.sfun_name = sfun_name

    def code_gen(self):
        # build code of system        
        res = TargetTemplate.code_gen(self)
        
        # get the system manifest and c++ class name
        m = res['manifest']
        simulation_name = m['api_name']
        
        # headers
        headers = '#include <stdio.h>\n'
        
        # define structs for I/O
        io_variables_define = simulation_name + '::Inputs inputs;\n' + simulation_name + '::Outputs outputs;\n'
        
        
        
        main_fn = """

/*  File    : sfun_counter_cpp.cpp
 *  Abstract:
 *
 *      Example of an C++ S-function which stores an C++ object in
 *      the pointers vector PWork.
 *
 *  Copyright 1990-2013 The MathWorks, Inc.
 */

#include <iostream>

""" + headers + '\n' + res['algorithm_sourcecode'] + """


class  counter {
    double  x;
public:
    counter() {
        x = 0.0;
    }
    double output(void) {
        x = x + 1.0;
        return x; 
    }
    double getX() const
    {
        return x;
    }
    void setX(double v)
    {
        x = v;
    }
};

#define S_FUNCTION_LEVEL 2
#define S_FUNCTION_NAME  """ + self.sfun_name + """

/*
 * Need to include simstruc.h for the definition of the SimStruct and
 * its associated macro definitions.
 */
#include "simstruc.h"

#define IS_PARAM_DOUBLE(pVal) (mxIsNumeric(pVal) && !mxIsLogical(pVal) &&\
!mxIsEmpty(pVal) && !mxIsSparse(pVal) && !mxIsComplex(pVal) && mxIsDouble(pVal))

/*====================*
 * S-function methods *
 *====================*/

#define MDL_CHECK_PARAMETERS
#if defined(MDL_CHECK_PARAMETERS)  && defined(MATLAB_MEX_FILE)
/*
 * Check to make sure that each parameter is 1-d and positive
 */
static void mdlCheckParameters(SimStruct *S)
{

    const mxArray *pVal0 = ssGetSFcnParam(S,0);

    if ( !IS_PARAM_DOUBLE(pVal0)) {
        ssSetErrorStatus(S, "Parameter to S-function must be a double scalar");
        return;
    } 
}
#endif


/* Function: mdlInitializeSizes ===============================================
 * Abstract:
 *    The sizes information is used by Simulink to determine the S-function
 *    block's characteristics (number of inputs, outputs, states, etc.).
 */
static void mdlInitializeSizes(SimStruct *S)
{
    ssSetNumSFcnParams(S, 1);  /* Number of expected parameters */
#if defined(MATLAB_MEX_FILE)
    if (ssGetNumSFcnParams(S) == ssGetSFcnParamsCount(S)) {
        mdlCheckParameters(S);
        if (ssGetErrorStatus(S) != NULL) {
            return;
        }
    } else {
        return; /* Parameter mismatch will be reported by Simulink */
    }
#endif
    ssSetSFcnParamTunable(S, 0, 0);

    ssSetNumContStates(S, 0);
    ssSetNumDiscStates(S, 0);

    if (!ssSetNumInputPorts(S, 0)) return;
    
    if (!ssSetNumOutputPorts(S, 1)) return;
    ssSetOutputPortWidth(S, 0, 1);

    ssSetNumSampleTimes(S, 1);
    ssSetNumRWork(S, 0);
    ssSetNumIWork(S, 0);
    ssSetNumPWork(S, 1); // reserve element in the pointers vector
    ssSetNumModes(S, 0); // to store a C++ object
    ssSetNumNonsampledZCs(S, 0);

    ssSetOperatingPointCompliance(S, USE_CUSTOM_OPERATING_POINT);

    ssSetOptions(S, 0);
}



/* Function: mdlInitializeSampleTimes =========================================
 * Abstract:
 *    This function is used to specify the sample time(s) for your
 *    S-function. You must register the same number of sample times as
 *    specified in ssSetNumSampleTimes.
 */
static void mdlInitializeSampleTimes(SimStruct *S)
{
    ssSetSampleTime(S, 0, mxGetScalar(ssGetSFcnParam(S, 0)));
    ssSetOffsetTime(S, 0, 0.0);
    ssSetModelReferenceSampleTimeDefaultInheritance(S);
}

#define MDL_START  /* Change to #undef to remove function */
#if defined(MDL_START) 
  /* Function: mdlStart =======================================================
   * Abstract:
   *    This function is called once at start of model execution. If you
   *    have states that should be initialized once, this is the place
   *    to do it.
   */
  static void mdlStart(SimStruct *S)
  {
      ssGetPWork(S)[0] = (void *) new """ + simulation_name + """; // store new C++ object in the
  }                                            // pointers vector
#endif /*  MDL_START */

/* Function: mdlOutputs =======================================================
 * Abstract:
 *    In this function, you compute the outputs of your S-function
 *    block.
 */
static void mdlOutputs(SimStruct *S, int_T tid)
{
    """ + simulation_name + """ *c = (""" + simulation_name + """ *) ssGetPWork(S)[0];
    
    
    real_T  *y = ssGetOutputPortRealSignal(S,0); // the pointers vector and use
    
    """ + io_variables_define + """
    
    inputs.u1 = 1;
    inputs.u2 = 2;
    
    c->step( outputs, inputs, true, false, false ); 
    
    y[0] = outputs.y1;
    
    UNUSED_ARG(tid);                             // object
}                                                


#define MDL_UPDATE
/* Function: mdlUpdate ========================================================
 * Abstract:
 *    This function is called once for every major integration time step.
 *    Discrete states are typically updated here, but this function is useful
 *    for performing any tasks that should only take place once per integration
 *    step.
 */
static void mdlUpdate(SimStruct *S, int_T tid)
{
    InputRealPtrsType uPtrs  = ssGetInputPortRealSignalPtrs(S,0);
    """ + simulation_name + """ *c = (""" + simulation_name + """ *) ssGetPWork(S)[0];
    """ + io_variables_define + """


    UNUSED_ARG(tid); /* not used in single tasking mode */
    
    inputs.u1 = 1; // U(i);
    inputs.u2 = 2;
    
    c->step( outputs, inputs, false, true, false ); 
}












/* Function: mdlTerminate =====================================================
 * Abstract:
 *    In this function, you should perform any actions that are necessary
 *    at the termination of a simulation.  For example, if memory was
 *    allocated in mdlStart, this is the place to free it.
 */
static void mdlTerminate(SimStruct *S)
{
    """ + simulation_name + """ *c = (""" + simulation_name + """ *) ssGetPWork(S)[0]; // retrieve and destroy C++
    delete c;                                  // object in the termination
}                                              // function
/*======================================================*
 * See sfuntmpl.doc for the optional S-function methods *
 *======================================================*/

/*=============================*
 * Required S-function trailer *
 *=============================*/

#ifdef  MATLAB_MEX_FILE    /* Is this file being compiled as a MEX-file? */
#include "simulink.c"      /* MEX-file interface mechanism */
#else
#include "cg_sfun.h"       /* Code generation registration function */
#endif


        
        """
        
        #
        # build main.cpp
        #
        main_cpp = main_fn
        
        cpp_fname = self.sfun_name + '.cpp'
        
        self.files[cpp_fname] = main_cpp
        
        # return
        return res

In [32]:
dy.clear()

system = dy.enter_system()

# define system inputs
u1               = dy.system_input( dy.DataTypeFloat64(1), name='u1', default_value=1.0,  value_range=[0, 25], title="input #1")
u2               = dy.system_input( dy.DataTypeFloat64(1), name='u2', default_value=2.0,  value_range=[0, 25], title="input #2")


y1 = dy.signal()                         # introduce variable y
x = y1 + u1                              # x[k] = y[k] + u[k]
y1 << dy.delay(x, initial_state = 2.0)   # y[k+1] = y[k] + x[k], y[0] = 2.0

# some second output
y2 = y1 + u2

# define output(s)
dy.append_output(y1, 'y1')
dy.append_output(y2, 'y2')

# generate code using the target defined above
code_gen_results = dy.generate_code(template=CppSimulinkSFunction(sfun_name='sfunction_demo'), folder='generated/simulink_sfunction')

compiling system simulation (level 0)... 
Generated code will be written to generated/simulink_sfunction .
writing file generated/simulink_sfunction/simulation_manifest.json
writing file generated/simulink_sfunction/sfunction_demo.cpp


### Generated manifest and source code 

In [18]:
# show the manifest

# from IPython.display import JSON, Code
# JSON(code_gen_results['manifest'], expanded=True)

In [19]:
# show the generated source code

Code(data=code_gen_results['files']['main.cpp'], language='c++')

/*  File    : sfun_counter_cpp.cpp
 *  Abstract:
 *
 *      Example of an C++ S-function which stores an C++ object in
 *      the pointers vector PWork.
 *
 *  Copyright 1990-2013 The MathWorks, Inc.
 */

#include <iostream>

#include <stdio.h>

// namespace for simulation {
  // global variables

  class simulation {
    public:


    // state update
    double block_2_mem;


    // state update


    //
    // cached output values
    //

    double y1__block_2;
    double y2__block_3;

    // API-function resetStates
    void resetStates() { // created by cpp_define_function

      block_2_mem = 2.0;
    }
    // output signals of  resetStates
    struct Outputs_resetStates{
      ;

    };
    // input signals of resetStates
    struct Inputs_resetStates{
      ;

    };
    // wrapper function for resetStates
    Outputs_resetStates resetStates__(Inputs_resetStates inputs) {
      Outputs_resetStates outputs;

      resetStates();

      return outputs;
    }
    // API-function updateStates
    void updateStates(double u1) { // created by cpp_define_function
      double s3;


      // restoring the signals y1, y2 from the states 
      double &y1 = y1__block_2;
      double &y2 = y2__block_3;


      // calculating the block outputs in the following order s3
      // that depend on u1
      // dependencies that require a state update are  

      s3 = y1 + u1;

      block_2_mem = s3;

      // calculating the block outputs in the following order 
      // that depend on 
      // dependencies that require a state update are  


    }
    // output signals of  updateStates
    struct Outputs_updateStates{
      ;

    };
    // input signals of updateStates
    struct Inputs_updateStates{
      double u1;

    };
    // wrapper function for updateStates
    Outputs_updateStates updateStates__(Inputs_updateStates inputs) {
      Outputs_updateStates outputs;

      updateStates(inputs.u1);

      return outputs;
    }
    // API-function calcResults_1 to compute: y1, y2
    void calcResults_1(double &y1, double &y2, double u2) { // created by cpp_define_function


      // calculating the block outputs in the following order y1, y2
      // that depend on u2
      // dependencies that require a state update are s3 

      y1 = block_2_mem;
      y2 = y1 + u2;

      // saving the signals y1, y2 into the states 
      y1__block_2 = y1;
      y2__block_3 = y2;
    }
    // output signals of  calcResults_1
    struct Outputs_calcResults_1{
      double y1;
      double y2;

    };
    // input signals of calcResults_1
    struct Inputs_calcResults_1{
      double u2;

    };
    // wrapper function for calcResults_1
    Outputs_calcResults_1 calcResults_1__(Inputs_calcResults_1 inputs) {
      Outputs_calcResults_1 outputs;

      calcResults_1(outputs.y1, outputs.y2,   inputs.u2);

      return outputs;
    }
    // all system inputs and outputs combined
    struct Inputs{
      double u1;
      double u2;

    };
    struct Outputs{
      double y1;
      double y2;

    };
    // main step function 
    void step(Outputs & outputs, Inputs const & inputs, int calculate_outputs, bool update_states, bool reset_states) {
      if (reset_states) {
        resetStates();

      }
      if (calculate_outputs==1) {
        calcResults_1(outputs.y1, outputs.y2,   inputs.u2);

      }
      if (update_states) {
        updateStates(inputs.u1);

      }

    }
  };

// end of namespace for simulation




class  counter {
    double  x;
public:
    counter() {
        x = 0.0;
    }
    double output(void) {
        x = x + 1.0;
        return x; 
    }
    double getX() const
    {
        return x;
    }
    void setX(double v)
    {
        x = v;
    }
};

#define S_FUNCTION_LEVEL 2
#define S_FUNCTION_NAME  sfun_counter_cpp

/*
 * Need to include simstruc.h for the definition of the SimStruct and
 * its associated macro definitions.
 */
#include "simstruc.h"

#define IS_PARAM_DOUBLE(pVal) (mxIsNumeric(pVal) && !mxIsLogical(pVal